In [11]:
import ollama as ol
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings,PromptTemplate
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.readers.file import PandasCSVReader
import warnings
warnings.filterwarnings("ignore")

In [4]:
#reading the patent dataset 
csv_reader = PandasCSVReader(concat_rows=False)
documents = csv_reader.load_data("ai_patents.csv" )

In [5]:
Settings.embed_model = OllamaEmbedding( model_name="nomic-embed-text:latest")
index = VectorStoreIndex.from_documents(documents, show_progress=True)

c:\Users\92343\AppData\Local\pypoetry\Cache\virtualenvs\writing-a-good-prompt-4KuF9HQv-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 1000/1000 [03:07<00:00,  5.32it/s]


In [32]:
Settings.llm = Ollama(model="llama3.2:1b", request_timeout=360.0)

qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    """You are an expert conversational patent chatbot that helps people with patents. 
    
    Your task:
    Create a response for the query provided, use the "context information" if its a question related to patents.

    Rules:
    If the query is related to patents then use the "Context Information" to create a concise and meaningful answer.
    Do not use any prior knowledge to create reponse for the query. 
    If it is a question and it is not about patents politely say you only help with patents.
    Do not mention that "context information" is provided. 

    
    -------------------------------------------
    Here are some good examples:

    Query: How to make pancakes?
    Answer: Sorry! I can not help you there. But feel free to ask me anything about patents!

    Query: Can you find patents that were pivotal in the development of digital twin analytics and vibration analysis? 
    Response: The invention leverages digital twin analytics and vibration analysis to predict X-ray tube liquid bearing failures, 
            ensuring reliable operation in medical imaging and industrial X-ray systems. By detecting vibrations in free-run mode,
            processing them to extract "coast down characteristics," and relaying the data to a cloud infrastructure, 
            it provides precise failure predictions. This innovation is pivotal for predictive maintenance, 
            reducing downtime, extending equipment lifespan, and enhancing operational efficiency in critical fields like healthcare and non-destructive testing. 
            Its integration of real-time monitoring and scalable cloud analytics marks a significant advancement in the field.

    Here are some bad examples:

    Query: hello! tell me what is 2+2?
    Response: 2+2 is 4.

    Query: Can you tell me about the patent Actively Stabilized Random Number Generator?
    Response: I did not recieve any context about it but I will use my prior knowldge to answer it.
    -----------------------------------

    Be concise and a good conversationalist.
     """
    "here is the query .\n"
    "Query: {query_str}\n"
    "Response: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine = index.as_query_engine(
    text_qa_template=qa_prompt_tmpl ,
    similarity_top_k = 3
)

In [33]:
while True:
    question = input()
    print("Me: ",question)
    if question.lower() == "exit": 
        break
    print("LLM: ",query_engine.query(question).response)

Me:  hi
LLM:  Hello. What can I help you with?
Me:  Can you tell me about patents related to AI?
LLM:  Patents related to artificial intelligence (AI) often involve innovative solutions for machine learning, natural language processing, computer vision, or other AI-specific technologies. Some examples of patents related to AI include:

- Methods and systems for improving the performance of AI models by leveraging various techniques such as regularization, data augmentation, and transfer learning.
- Devices and platforms that utilize AI for tasks like image recognition, speech recognition, and predictive maintenance in industries like healthcare, finance, and manufacturing.
- Software applications that provide AI-driven services for customer service, content generation, or recommendation systems.

These are just a few examples of patents related to AI. If you're looking for specific information on a particular patent or topic, feel free to ask!
Me:  how to make pancakes?
LLM:  Sorry, I 